# ETL Pipeline Results

## Analytical Tables (DWH) Contents

In [49]:
from configparser import ConfigParser
from sqlalchemy import create_engine
import pandas as pd
import unittest
import math

config = ConfigParser()
config.read('./../dwh.cfg')

HOST = config.get('CLUSTER', 'HOST')
DB_NAME = config.get('CLUSTER', 'DB_NAME')
DB_USER = config.get('CLUSTER', 'DB_USER')
DB_PASSWORD = config.get('CLUSTER', 'DB_PASSWORD')
DB_PORT = config.get('CLUSTER', 'DB_PORT')

conn_string = f"postgresql://{DB_USER}:{DB_PASSWORD}@{HOST}:{DB_PORT}/{DB_NAME}"
conn = create_engine(conn_string, client_encoding="UTF-8")

## Check Analytical Tables Loaded Correctly

We have the following Analytical Tables:
* songplays
* users
* artists
* songs
* time

And these are the staging tables:
* staging_songs
* staging_events

Before continue executing sample analytical queries it is better to check DWH table rows contain the expected values.

First lets check the table counts:

In [50]:
pd.read_sql(f"""
SELECT 'staging_songs' as table_name, count(*) as count from staging_songs
UNION SELECT 'staging_events' as table_name, count(*) as count from staging_events where page='NextSong'
UNION SELECT 'songplays' as table_name, count(*) as count from songplays
UNION SELECT 'users' as table_name, count(*) as count from users
UNION SELECT 'artists' as table_name, count(*) as count from artists
UNION SELECT 'songs' as table_name, count(*) as count from songs
UNION SELECT 'time' as table_name, count(*) as count from time;
""", con = conn)

,table_name,count
0,staging_events,6820
1,songs,5103
2,time,6486
3,staging_songs,385252
4,songplays,6606
5,artists,3040
6,users,96


## Check The Data for a Specific Criteria

As a methodology, to check the DWH table's contents are the same as our expectations, we will define a set of criteria.

The selected search criteria are the same that we have used on `analyze_staging_data.ipynb`:

In [51]:
artist = 'Harmonia'
song = 'Sehr kosmisch'
length = '655.77751'

Firstly the query results are stored, then we will iterate each row and compare the rows.

In [52]:
df_staging_events = pd.read_sql(f"""
SELECT * FROM staging_events
WHERE page='NextSong' and artist = '{artist}' and song = '{song}' and length = {length}
ORDER BY ts, user_id;
""", con=conn)

df_staging_events.head(100)

,artist,auth,first_name,gender,item_in_session,last_name,length,level,location,method,page,registration,session_id,song,status,ts,user_agent,user_id
0,Harmonia,Logged In,Harper,M,22,Barrett,655.77751,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1540685364796,129,Sehr kosmisch,200,1541411918796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42
1,Harmonia,Logged In,Kate,F,48,Harrell,655.77751,paid,"Lansing-East Lansing, MI",PUT,NextSong,1540472624796,293,Sehr kosmisch,200,1541539531796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97
2,Harmonia,Logged In,Aleena,F,0,Kirby,655.77751,paid,"Waterloo-Cedar Falls, IA",PUT,NextSong,1541022995796,285,Sehr kosmisch,200,1541596896796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44
3,Harmonia,Logged In,Hayden,F,10,Brock,655.77751,paid,"Detroit-Warren-Dearborn, MI",PUT,NextSong,1540852600796,117,Sehr kosmisch,200,1541675751796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,72
4,Harmonia,Logged In,Stefany,F,0,White,655.77751,free,"Lubbock, TX",PUT,NextSong,1540708070796,380,Sehr kosmisch,200,1542100680796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
5,Harmonia,Logged In,Chloe,F,14,Cuevas,655.77751,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1540940782796,568,Sehr kosmisch,200,1542186926796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
6,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1541016707796,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
7,Harmonia,Logged In,Avery,F,55,Watkins,655.77751,paid,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1540871783796,324,Sehr kosmisch,200,1542288215796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30
8,Harmonia,Logged In,Ryan,M,2,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1541016707796,622,Sehr kosmisch,200,1542355762796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26
9,Harmonia,Logged In,Chloe,F,22,Cuevas,655.77751,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1540940782796,636,Sehr kosmisch,200,1542363544796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49


In [53]:
df_staging_songs = pd.read_sql(f"""
SELECT * FROM staging_songs
WHERE artist_name = '{artist}' and title = '{song}' and duration = {length};
""", con=conn)

df_staging_songs.head(100)

,song_id,num_songs,title,artist_name,artist_latitude,year,duration,artist_id,artist_longitude,artist_location
0,SOFRQTD12A81C233C0,1,Sehr kosmisch,Harmonia,34.05349,0,655.77751,AR0IVTL1187B9AD520,-118.24532,United States


With the defined criteria:
* artist = 'Harmonia'
* song = 'Sehr kosmisch'
* length = '655.77751'

We have:
* 21 records in `staging_events`
* 1 record in `staging_songs`

On following analytical query, we are expecting again the same 21 records that contain the same information queried above.

In [54]:
df_dwh_query_result = pd.read_sql(f"""
SELECT  sp.*, u.*, ar.*, s.*, t.*,
        sp.location as songsplay_location, ar.location as artist_location,
        s.year as songs_year, t.year as time_year,
        sp.level as songsplay_level
FROM songplays as sp
INNER JOIN users as u ON sp.user_id = u.user_id
INNER JOIN artists as ar ON sp.artist_id = ar.artist_id
INNER JOIN songs as s ON sp.song_id = s.song_id
INNER JOIN time as t ON sp.start_time = t.start_time
WHERE ar.name = '{artist}' and s.title = '{song}' and s.duration = {length}
ORDER BY sp.start_time, sp.user_id;
""", con=conn)

df_dwh_query_result.head(100)

,songplay_id,start_time,user_id,level,song_id,artist_id,session_id,location,user_agent,user_id,...,day,week,month,year,weekday,songsplay_location,artist_location,songs_year,time_year,songsplay_level
0,4595,2018-11-05 09:58:38,42,paid,SOFRQTD12A81C233C0,AR0IVTL1187B9AD520,129,"New York-Newark-Jersey City, NY-NJ-PA","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42,...,5,45,11,2018,1,"New York-Newark-Jersey City, NY-NJ-PA",United States,0,2018,paid
1,4593,2018-11-06 21:25:31,97,paid,SOFRQTD12A81C233C0,AR0IVTL1187B9AD520,293,"Lansing-East Lansing, MI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97,...,6,45,11,2018,2,"Lansing-East Lansing, MI",United States,0,2018,paid
2,4591,2018-11-07 13:21:36,44,paid,SOFRQTD12A81C233C0,AR0IVTL1187B9AD520,285,"Waterloo-Cedar Falls, IA",Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,44,...,7,45,11,2018,3,"Waterloo-Cedar Falls, IA",United States,0,2018,paid
3,4589,2018-11-08 11:15:51,72,paid,SOFRQTD12A81C233C0,AR0IVTL1187B9AD520,117,"Detroit-Warren-Dearborn, MI",Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,72,...,8,45,11,2018,4,"Detroit-Warren-Dearborn, MI",United States,0,2018,paid
4,4585,2018-11-13 09:18:00,83,free,SOFRQTD12A81C233C0,AR0IVTL1187B9AD520,380,"Lubbock, TX","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83,...,13,46,11,2018,2,"Lubbock, TX",United States,0,2018,free
5,4587,2018-11-14 09:15:26,49,paid,SOFRQTD12A81C233C0,AR0IVTL1187B9AD520,568,"San Francisco-Oakland-Hayward, CA",Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49,...,14,46,11,2018,3,"San Francisco-Oakland-Hayward, CA",United States,0,2018,paid
6,4577,2018-11-15 00:30:26,26,free,SOFRQTD12A81C233C0,AR0IVTL1187B9AD520,583,"San Jose-Sunnyvale-Santa Clara, CA","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,...,15,46,11,2018,4,"San Jose-Sunnyvale-Santa Clara, CA",United States,0,2018,free
7,4575,2018-11-15 13:23:35,30,paid,SOFRQTD12A81C233C0,AR0IVTL1187B9AD520,324,"San Jose-Sunnyvale-Santa Clara, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,30,...,15,46,11,2018,4,"San Jose-Sunnyvale-Santa Clara, CA",United States,0,2018,paid
8,4583,2018-11-16 08:09:22,26,free,SOFRQTD12A81C233C0,AR0IVTL1187B9AD520,622,"San Jose-Sunnyvale-Santa Clara, CA","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26,...,16,46,11,2018,5,"San Jose-Sunnyvale-Santa Clara, CA",United States,0,2018,free
9,4581,2018-11-16 10:19:04,49,paid,SOFRQTD12A81C233C0,AR0IVTL1187B9AD520,636,"San Francisco-Oakland-Hayward, CA",Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49,...,16,46,11,2018,5,"San Francisco-Oakland-Hayward, CA",United States,0,2018,paid


We have again 21 rows in the analytical query result.

Now let's check & test each query row values. Since we have already sorted the results by time & user, rows on `staging_events` query should match the rows on `songsplay` query.

In [55]:
# Needed to clear duplicated columns. Same named columns are added with a different names
df_dwh_query_result = df_dwh_query_result.loc[:, ~df_dwh_query_result.columns.duplicated()].copy()

# We have a single staging songs row
staging_songs_row = df_staging_songs.iloc[0]

tc = unittest.TestCase()
for index, query_row in df_dwh_query_result.iterrows():
    staging_events_row = df_staging_events.iloc[index]
    # Event Data Match Tests: songplays, users table's rows
    tc.assertEqual(staging_events_row['user_id'], query_row['user_id'])
    tc.assertEqual(staging_events_row['level'], query_row['songsplay_level'])  # songsplay_level is added to query
    tc.assertEqual(staging_events_row['gender'], query_row['gender'])
    tc.assertEqual(staging_events_row['first_name'], query_row['first_name'])
    tc.assertEqual(staging_events_row['last_name'], query_row['last_name'])
    tc.assertEqual(staging_events_row['session_id'], query_row['session_id'])
    tc.assertEqual(staging_events_row['location'], query_row['songsplay_location'])  # songsplay_location is added to query
    tc.assertEqual(staging_events_row['user_agent'], query_row['user_agent'])

    # Event Data Match Tests: time table's rows
    ts_second = math.floor(staging_events_row['ts'] / 1000)
    ts_timestamp = pd.Timestamp(ts_second, unit='s')

    tc.assertEqual(ts_timestamp.timestamp(), query_row['start_time'].timestamp())
    tc.assertEqual(ts_timestamp.hour, query_row['hour'])
    tc.assertEqual(ts_timestamp.day, query_row['day'])
    tc.assertEqual(ts_timestamp.hour, query_row['hour'])
    tc.assertEqual(ts_timestamp.week, query_row['week'])
    tc.assertEqual(ts_timestamp.month, query_row['month'])
    tc.assertEqual(ts_timestamp.year, query_row['time_year'])  # time_year is added to query

    # Song Data Match Tests: songplays, artists & songs table's rows
    tc.assertEqual(staging_songs_row['song_id'], query_row['song_id'])
    tc.assertEqual(staging_songs_row['title'], query_row['title'])
    tc.assertEqual(staging_songs_row['artist_name'], query_row['name'])
    tc.assertEqual(staging_songs_row['artist_latitude'], query_row['latitude'])
    tc.assertEqual(staging_songs_row['year'], query_row['songs_year'])   # songs_year is added to query
    tc.assertEqual(staging_songs_row['duration'], query_row['duration'])
    tc.assertEqual(staging_songs_row['artist_id'], query_row['artist_id'])
    tc.assertEqual(staging_songs_row['artist_longitude'], query_row['longitude'])
    tc.assertEqual(staging_songs_row['artist_location'], query_row['artist_location'])  # artist_location is added to query

    print(f"DWH query row {index+1} values successfully matched!")

DWH query row 1 values successfully matched!
DWH query row 2 values successfully matched!
DWH query row 3 values successfully matched!
DWH query row 4 values successfully matched!
DWH query row 5 values successfully matched!
DWH query row 6 values successfully matched!
DWH query row 7 values successfully matched!
DWH query row 8 values successfully matched!
DWH query row 9 values successfully matched!
DWH query row 10 values successfully matched!
DWH query row 11 values successfully matched!
DWH query row 12 values successfully matched!
DWH query row 13 values successfully matched!
DWH query row 14 values successfully matched!
DWH query row 15 values successfully matched!
DWH query row 16 values successfully matched!
DWH query row 17 values successfully matched!
DWH query row 18 values successfully matched!
DWH query row 19 values successfully matched!
DWH query row 20 values successfully matched!
DWH query row 21 values successfully matched!


Our DWH tables are successfully created & inserted!